In [ ]:
import operator
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

TRAIN_DIR = '../res/Proper'
IMG_SIZE_X = 160
IMG_SIZE_Y = 120
LR = 1e-3
labels = os.listdir(TRAIN_DIR)
LABELS_COUNT = len(labels)
MODEL_NAME = 'cactusifier-{}-{}.model'.format(LR, '2conv-basic')

labelCounts = {}
for directory in labels:
    files = os.listdir(os.path.join(TRAIN_DIR, directory))
    labelCounts[directory] = len(files)

print(sum(labelCounts.values()), 'images')
labelCounts = sorted(labelCounts.items(), key=operator.itemgetter(1), reverse = True)
labelCounts

In [ ]:
def get_hot_one_encoded_label(label):
    h1e = np.zeros(LABELS_COUNT)
    h1e[labels.index(label)] = 1
    return h1e

In [ ]:
def get_label_from_h1e(h1e):
    index = np.where(h1e == 1)[0][0]
    return labels[index]

In [ ]:
def get_label_from_index(index):
    return labels[index]

In [ ]:
def create_train_data():
    training_data = []
    counter = 1
    for directory in labels:
        print(counter, directory)
        labelDirectory = os.path.join(TRAIN_DIR, directory)
        files = os.listdir(labelDirectory)
        for img in files:
            label = get_hot_one_encoded_label(directory)
            path = os.path.join(labelDirectory, img)
            try:
                img = cv2.resize(cv2.imread(path, cv2.IMREAD_COLOR), (IMG_SIZE_X, IMG_SIZE_Y))
            except Exception as e:
                print(path)
                print(str(e))
            training_data.append([np.array(img), np.array(label)])
        counter = counter + 1
    print('Shuffling')
    shuffle(training_data)
    print('Saving')
    #np.save('train_data.npy', training_data)
    print('Done!')
    return training_data

In [ ]:
train_data = create_train_data()

In [ ]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensorflow as tf
tf.reset_default_graph()

convnet = input_data(shape=[None, IMG_SIZE_X, IMG_SIZE_Y, 3], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, LABELS_COUNT, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


In [ ]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print("Model", MODEL_NAME, "loaded!")

In [ ]:
trainCountRatio = 0.8
trainSize = int(round(len(train_data)*trainCountRatio))
test = train_data[:-trainSize]
train = train_data[-trainSize:]

In [ ]:
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE_X, IMG_SIZE_Y, 3)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE_X, IMG_SIZE_Y, 3)
test_y = [i[1] for i in test]

In [ ]:
model.fit({'input': X}, {'targets': Y}, n_epoch=15, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

In [ ]:
model.save(MODEL_NAME)

In [ ]:
testImage = np.array(cv2.resize(cv2.imread('T01.jpg', cv2.IMREAD_COLOR), (IMG_SIZE_X, IMG_SIZE_Y)))
testImage = testImage.reshape(-1, IMG_SIZE_X, IMG_SIZE_Y, 3)
result = model.predict(testImage)
labels[np.argmax(result)]


In [ ]:
s = sorted(enumerate(result[0]), key=lambda x: x[1])
for probability in s:
    print(get_label_from_index(probability[0]))

In [ ]:
#tensorboard --logdir=foo:E:\Programming\Python\Cactusifier\app\log